# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import os
import utils
import TD3

import wandb

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='/home/corey/reinforcement-learning/udacity/deep-reinforcement-learning/p2_continuous-control/Reacher_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
timesteps = 0
while True:
    timesteps += 1
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
env_info = env.reset(train_mode=True)[brain_name]

# TD3 with reacher env

In [6]:

# Hyperparameters
policy ="TD3"                 # Policy name (TD3, DDPG or OurDDPG)
seed = 0               # Sets Gym, PyTorch and Numpy seeds
start_timesteps = 25e3# Time steps initial random policy is used
max_timesteps = 329_329
# max_timesteps = 50_000
eval_freq = 5e3      # How often (time steps) we evaluate
# max_timesteps = int(1e6)   # Max time steps to run environment
expl_noise = 0.1              # Std of Gaussian exploration noise
batch_size = 256      # Batch size for both actor and critic
discount = 0.99                 # Discount factor
tau = 0.005                     # Target network update rate
policy_noise = 0.2              # Noise added to target policy during critic update
noise_clip = 0.5                # Range to clip target policy noise
policy_freq = 2       # Frequency of delayed policy updates
save_model = True        # Save model and optimizer parameters
load_model = ""                # Model load file name, "" doesn't load, "default" uses file_name

In [7]:

wandb.init(project="Continuous-Control")

wandb.config = {
    "policy": policy,
    "seed": seed, 
    #TODO add the rest
}

if not os.path.exists("./results"):
    os.makedirs("./results")

if save_model and not os.path.exists("./models"):
    os.makedirs("./models")
torch.manual_seed(seed)
np.random.seed(seed)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: coreymorris. Use `wandb login --relogin` to force relogin


In [8]:
env_info = env.reset(train_mode=True)[brain_name]

action_size = brain.vector_action_space_size

states = env_info.vector_observations
state_size = states.shape[1]

state_dim = state_size
action_dim = action_size
max_action = 1.0

In [9]:
kwargs = {
    "state_dim": state_dim,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": discount,
    "tau": tau,
}

# Initialize policy
if policy == "TD3":
    # Target policy smoothing is scaled wrt the action scale
    kwargs["policy_noise"] = policy_noise * max_action
    kwargs["noise_clip"] = noise_clip * max_action
    kwargs["policy_freq"] = policy_freq
    policy = TD3.TD3(**kwargs)

if load_model != "":
    policy_file = file_name if load_model == "default" else load_model
    policy.load(f"./models/{policy_file}")

replay_buffer = utils.ReplayBuffer(state_dim, action_dim)

In [10]:
# # Evaluate untrained policy
# evaluations = [eval_policy(policy, env_name, seed)]

env_info = env.reset(train_mode=True)[brain_name] 
state = env_info.vector_observations[0]
done = env_info.local_done[0]
  
episode_reward = 0
episode_timesteps = 0
episode_num = 0

wandb.watch(policy.actor, log_freq=100)

[]

In [11]:

print_every = 100
scores_deque = deque(maxlen=print_every)
scores = []
for t in range(int(max_timesteps)):
    
    episode_timesteps += 1

    # Select action randomly or according to policy
    if t < start_timesteps:
        action = np.random.rand(1, action_size)
    else:
        action = (
            policy.select_action(np.array(state))
            + np.random.normal(0, max_action * expl_noise, size=action_dim)
        ).clip(-max_action, max_action)

    # Perform action
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]  
    done_bool = float(done) if env_info.max_reached[0] else 0

    # Store data in replay buffer
    replay_buffer.add(state, action, next_state, reward, done_bool)

    state = next_state
    episode_reward += reward

    # Train agent after collecting sufficient data
    if t >= start_timesteps:
        policy.train(replay_buffer, batch_size)

    if done: 
        # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
        wandb.log({"episode number": episode_num + 1})
        wandb.log({"episode timesteps": episode_timesteps})
        wandb.log({"episode reward": episode_reward})
        wandb.log({"total timesteps": t + 1})
        
        
        print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
        # Reset environment
        env_info = env.reset(train_mode=True)[brain_name] 
        state = env_info.vector_observations[0]
        done = env_info.local_done[0]
        scores.append(episode_reward)
        scores_deque.append(episode_reward)  
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1
        if episode_num % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode_num, np.mean(scores_deque)))
            wandb.log({"Average Reward over last 100 episodes": np.mean(scores_deque)})

wandb.finish()


Total T: 1001 Episode Num: 1 Episode T: 1001 Reward: 0.000
Total T: 2002 Episode Num: 2 Episode T: 1001 Reward: 0.000
Total T: 3003 Episode Num: 3 Episode T: 1001 Reward: 0.000
Total T: 4004 Episode Num: 4 Episode T: 1001 Reward: 0.000
Total T: 5005 Episode Num: 5 Episode T: 1001 Reward: 0.000
Total T: 6006 Episode Num: 6 Episode T: 1001 Reward: 0.000
Total T: 7007 Episode Num: 7 Episode T: 1001 Reward: 0.000
Total T: 8008 Episode Num: 8 Episode T: 1001 Reward: 0.000
Total T: 9009 Episode Num: 9 Episode T: 1001 Reward: 0.000
Total T: 10010 Episode Num: 10 Episode T: 1001 Reward: 0.000
Total T: 11011 Episode Num: 11 Episode T: 1001 Reward: 0.000
Total T: 12012 Episode Num: 12 Episode T: 1001 Reward: 0.000
Total T: 13013 Episode Num: 13 Episode T: 1001 Reward: 0.000
Total T: 14014 Episode Num: 14 Episode T: 1001 Reward: 0.000
Total T: 15015 Episode Num: 15 Episode T: 1001 Reward: 0.000
Total T: 16016 Episode Num: 16 Episode T: 1001 Reward: 0.000
Total T: 17017 Episode Num: 17 Episode T: 

Average Reward over last 100 episodes,▁▇█
episode number,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode reward,▁▁▁▁▁▁▂▂▃▃▄▄▄▅▇█▇▇▇█▇█▆██▇████▇▇▇█▇██▇██
episode timesteps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total timesteps,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Average Reward over last 100 episodes,36.8623
episode number,329
episode reward,39.44
episode timesteps,1001
total timesteps,329329


In [12]:
policy.save(f"./models/308_reacher_3")